In [184]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import xgboost as xgb
import sklearn
from tensorflow import keras
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_classification
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
import pickle

In [185]:
datasets = {chr(letter):pd.read_csv(f'processed_data_mp01021/{chr(letter)}_Data.csv') for letter in range(ord('A'), ord('Z') + 1)}

In [186]:
datasets['A'].head()

,Landmarks,0,0.1,0.2,1,1.1,1.2,2,2.1,2.2,...,17.2,18,18.1,18.2,19,19.1,19.2,20,20.1,20.2
0,Coordinates,x,y,z,x,y,z,x,y,z,...,z,x,y,z,x,y,z,x,y,z
1,Frame,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,-0.10191626101732254,0.9925596117973328,0.06662236154079437,0.2784864604473114,0.8289384841918945,-0.07882609963417053,0.6251492500305176,0.3535401523113251,-0.11874441802501678,...,-0.09184916317462921,-0.28185784816741943,0.07575207948684692,-0.28906363248825073,-0.2340974658727646,0.4078732132911682,-0.257875919342041,-0.22813771665096283,0.6210938096046448,-0.17181435227394104
3,2,-0.11209727078676224,0.99217689037323,0.05494748428463936,0.25864800810813904,0.7781158685684204,-0.08616439998149872,0.5868735313415527,0.33302342891693115,-0.13564430177211761,...,-0.07887183874845505,-0.29231786727905273,0.06680270284414291,-0.23227861523628235,-0.2517852783203125,0.38482457399368286,-0.17785505950450897,-0.248565673828125,0.5935962796211243,-0.07973474264144897
4,3,0.025396086275577545,0.9988321661949158,0.04110104590654373,0.3819988965988159,0.7106916308403015,-0.09989170730113983,0.6305642127990723,0.20811787247657776,-0.14538881182670593,...,-0.09832348674535751,-0.27134260535240173,0.0629710778594017,-0.28379887342453003,-0.20002099871635437,0.370392769575119,-0.237660214304924,-0.16897597908973694,0.6012681722640991,-0.13964223861694336


In [187]:
for key, value in datasets.items():
    datasets[key] = datasets[key].drop([0,1])
    datasets[key] = datasets[key].drop(columns=['Landmarks'])
    datasets[key]['Letter'] = key
    datasets[key] = datasets[key].sample(frac=1).reset_index(drop=True)

In [188]:
datasets['B'].head()

,0,0.1,0.2,1,1.1,1.2,2,2.1,2.2,3,...,18,18.1,18.2,19,19.1,19.2,20,20.1,20.2,Letter
0,-0.2207265943288803,0.9706849455833435,0.09513436257839203,0.021739261224865913,0.7742661833763123,-0.08472975343465805,0.17241446673870087,0.44143056869506836,-0.14812597632408142,0.006798033602535725,...,-0.3365214765071869,-0.1492941677570343,-0.11200330406427383,-0.31004250049591064,-0.3395440876483917,-0.15412279963493347,-0.2848142981529236,-0.5114002227783203,-0.18586255609989166,B
1,-0.20247018337249756,0.9789202809333801,0.026849787682294846,0.05141550675034523,0.7501966953277588,-0.07845427095890045,0.18660153448581696,0.37376949191093445,-0.09688098728656769,0.06067362427711487,...,-0.29687100648880005,-0.1418548822402954,-0.13494262099266052,-0.28655076026916504,-0.3353334963321686,-0.16880613565444946,-0.2958759069442749,-0.5093653798103333,-0.19815412163734436,B
2,-0.07129079103469849,0.9955935478210449,0.060919132083654404,0.15722773969173431,0.7893714904785156,-0.06470795720815659,0.268802672624588,0.4028640389442444,-0.09475712478160858,0.12796550989151,...,-0.2835652828216553,-0.07572918385267258,-0.15866991877555847,-0.30369505286216736,-0.26507437229156494,-0.2053084671497345,-0.34527119994163513,-0.4360535144805908,-0.23940931260585785,B
3,-0.1520264744758606,0.9869832396507263,0.052460405975580215,0.09376238286495209,0.7781801819801331,-0.07616899162530899,0.24171659350395203,0.3992753326892853,-0.1033237874507904,0.10847577452659607,...,-0.3137594759464264,-0.10752419382333755,-0.1442088633775711,-0.3078997731208801,-0.29551517963409424,-0.18990564346313477,-0.31387466192245483,-0.4701763093471527,-0.22609838843345642,B
4,-0.2542685270309448,0.9648824334144592,0.06594978272914886,0.008232724852859974,0.7797635197639465,-0.04831967502832413,0.17219723761081696,0.4189990758895874,-0.07970911264419556,0.07978304475545883,...,-0.29054975509643555,-0.139352485537529,-0.15138418972492218,-0.27102336287498474,-0.3216945230960846,-0.1821233183145523,-0.2713964283466339,-0.488657146692276,-0.20901627838611603,B


In [189]:
data = pd.concat(datasets.values(), ignore_index=True)
data = data.sample(frac=1).reset_index(drop=True)
data.head()

,0,0.1,0.2,1,1.1,1.2,2,2.1,2.2,3,...,18,18.1,18.2,19,19.1,19.2,20,20.1,20.2,Letter
0,-0.08142708241939545,0.9923756718635559,0.09252084046602249,0.22144319117069244,0.7998363375663757,0.11282114684581757,0.4199620485305786,0.508604884147644,0.08388601243495941,0.5590741634368896,...,-0.22382524609565735,-0.18120138347148895,-0.3240731656551361,-0.03931759297847748,-0.3214283287525177,-0.3725730776786804,0.13631491363048553,-0.32833319902420044,-0.3902526795864105,C
1,-0.018885105848312378,0.9858821630477905,0.16637232899665833,0.251800537109375,0.7381811738014221,0.028740810230374336,0.3737737238407135,0.29367053508758545,-0.02712497115135193,0.2091236263513565,...,-0.20898529887199402,0.06275954097509384,-0.29118335247039795,-0.08059407025575638,0.25093957781791687,-0.2752704322338104,-0.06930302828550339,0.4208160638809204,-0.21231234073638916,U
2,-0.4363015294075012,0.8898440599441528,0.13348591327667236,-0.16355547308921814,0.7757006883621216,-0.046483363956213,0.08270754665136337,0.5467744469642639,-0.1437119096517563,0.1164417713880539,...,-0.41686439514160156,-0.060270775109529495,-0.2210739403963089,-0.39340919256210327,0.137828066945076,-0.17634929716587067,-0.3785313367843628,0.26437342166900635,-0.09646275639533997,Z
3,0.5751295685768127,0.8037475943565369,0.15236707031726837,0.6690853834152222,0.4786679148674011,-0.0041204215958714485,0.5347392559051514,0.11140614002943039,-0.06322501599788666,0.2505123019218445,...,0.017967453226447105,0.3368670642375946,-0.27844661474227905,0.2501538395881653,0.39492034912109375,-0.2933789789676666,0.4171738922595978,0.4737691879272461,-0.24470023810863495,H
4,-0.2935364842414856,0.9515162110328674,0.09194131195545197,-0.033419910818338394,0.7704428434371948,-0.09141205251216888,0.1522606611251831,0.4557228684425354,-0.15609867870807648,0.020121047273278236,...,-0.32727354764938354,-0.18368437886238098,-0.1099257692694664,-0.28734949231147766,-0.37075719237327576,-0.15213990211486816,-0.249105766415596,-0.5426187515258789,-0.1828824132680893,B


In [190]:
y = data['Letter']
X = data.drop(columns=['Letter'])

In [191]:
X_trainval, X_test, y_trainval, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_trainval, y_trainval, test_size=0.3, random_state=42)

# scaler = StandardScaler()
# X_train = scaler.fit_transform(X_train)
# X_val = scaler.transform(X_val)
# X_test = scaler.transform(X_test)
X_train = X_train.apply(pd.to_numeric, errors='coerce')
X_train = X_train.dropna()
X_train = X_train.to_numpy()
X_val = X_val.apply(pd.to_numeric, errors='coerce')
X_val = X_val.dropna()
X_val = X_val.to_numpy()
X_test = X_test.apply(pd.to_numeric, errors='coerce')
X_test = X_test.dropna()
X_test = X_test.to_numpy()


le = LabelEncoder()
y_train = le.fit_transform(y_train)  
y_val = le.transform(y_val)  
y_test = le.transform(y_test)

In [207]:
X_train.shape

(14728, 63)

In [193]:
# Define model function
def create_keras_model():
    model = keras.Sequential([
        keras.layers.Dense(128, activation='relu', input_shape=(X_train.shape[1],)),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.3),  # Helps prevent overfitting

        keras.layers.Dense(64, activation='relu'),
        keras.layers.BatchNormalization(),
        keras.layers.Dropout(0.3),

        keras.layers.Dense(32, activation='relu'),

        keras.layers.Dense(26, activation='softmax')  # 26 classes
    ])

    # Compile the model
    model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',  # Use if labels are integers (0-25)
                metrics=['accuracy'])
    return model


In [194]:
keras_model = create_keras_model()
keras_model.fit(X_train, y_train, epochs=20, batch_size=32, verbose=0, validation_data=(X_val, y_val))

d:\ASL-Fingerspelling-Recognition\.venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [195]:
val_loss, val_acc = keras_model.evaluate(X_val, y_val, verbose=0)
print(f"Final Validation Accuracy: {val_acc:.4f}")

Final Validation Accuracy: 0.9967


In [196]:
test_loss, test_acc = keras_model.evaluate(X_test, y_test, verbose=0)
print(f"Final Test Accuracy: {test_acc:.4f}")

Final Test Accuracy: 0.9973


In [197]:
keras_model.save("models/keras_ANN.keras")

In [198]:
def create_xgb_model():
    model = xgb.XGBClassifier(
        objective="multi:softmax",  # Multi-class classification
        num_class=26,               # 26 different classes
        eval_metric="mlogloss",      # Multiclass log loss
        max_depth=6,                 # Depth of trees (adjust based on experiments)
        learning_rate=0.1,           # Step size shrinkage (adjust for tuning)
        n_estimators=100,            # Number of trees (increase for better results)
        use_label_encoder=False      # Required for newer versions of XGBoost
    )
    return model

In [199]:
xgb_model = create_xgb_model()
xgb_model.fit(X_train, y_train, eval_set=[(X_val, y_val)], verbose=True)

[0]	validation_0-mlogloss:2.09876
[1]	validation_0-mlogloss:1.73070


d:\ASL-Fingerspelling-Recognition\.venv\Lib\site-packages\xgboost\core.py:158: UserWarning: [23:11:53] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-08cbc0333d8d4aae1-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


[2]	validation_0-mlogloss:1.47860
[3]	validation_0-mlogloss:1.28654
[4]	validation_0-mlogloss:1.13236
[5]	validation_0-mlogloss:1.00366
[6]	validation_0-mlogloss:0.89470
[7]	validation_0-mlogloss:0.80094
[8]	validation_0-mlogloss:0.71922
[9]	validation_0-mlogloss:0.64742
[10]	validation_0-mlogloss:0.58404
[11]	validation_0-mlogloss:0.52768
[12]	validation_0-mlogloss:0.47744
[13]	validation_0-mlogloss:0.43262
[14]	validation_0-mlogloss:0.39246
[15]	validation_0-mlogloss:0.35652
[16]	validation_0-mlogloss:0.32410
[17]	validation_0-mlogloss:0.29496
[18]	validation_0-mlogloss:0.26857
[19]	validation_0-mlogloss:0.24479
[20]	validation_0-mlogloss:0.22324
[21]	validation_0-mlogloss:0.20384
[22]	validation_0-mlogloss:0.18632
[23]	validation_0-mlogloss:0.17048
[24]	validation_0-mlogloss:0.15599
[25]	validation_0-mlogloss:0.14294
[26]	validation_0-mlogloss:0.13112
[27]	validation_0-mlogloss:0.12034
[28]	validation_0-mlogloss:0.11053
[29]	validation_0-mlogloss:0.10160
[30]	validation_0-mlogloss:0

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.1, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_class=26, num_parallel_tree=None, ...)

In [200]:
# Make predictions
y_pred_val = xgb_model.predict(X_val)

# Compute accuracy
accuracy = accuracy_score(y_val, y_pred_val)
print(f"Evaluation Accuracy: {accuracy:.4f}")

Evaluation Accuracy: 0.9978


In [201]:
# Make predictions
y_pred_test = xgb_model.predict(X_test)

# Compute accuracy
accuracy = accuracy_score(y_test, y_pred_test)
print(f"Test Accuracy: {accuracy:.4f}")

Test Accuracy: 0.9970


In [202]:
xgb_model.save_model("models/xgboost_model.json")

In [203]:
X_test

array([[-1.02237448e-01,  9.89239454e-01,  1.04655616e-01, ...,
         1.74758155e-02,  3.82115394e-01, -2.23726198e-01],
       [ 8.40613768e-02,  9.83648717e-01,  1.59276232e-01, ...,
         2.97183084e-04,  5.75120151e-01, -2.29534343e-01],
       [-2.50038236e-01,  9.68009591e-01, -2.09370777e-02, ...,
        -4.39779162e-01, -8.43960881e-01, -1.76155865e-01],
       ...,
       [-9.67272893e-02,  9.87501383e-01,  1.24438114e-01, ...,
         2.39147376e-02,  3.89859766e-01, -1.93353578e-01],
       [-1.09583504e-01,  9.92640495e-01,  5.15393503e-02, ...,
        -3.62617791e-01,  3.03899199e-01,  9.57416545e-04],
       [ 9.88415837e-01,  3.68006006e-02,  1.47240788e-01, ...,
         1.91839971e-02,  7.13500738e-01, -1.13028035e-01]])

In [204]:
X_test = X_test[0].reshape(1, -1)
X_test

array([[-0.10223745,  0.98923945,  0.10465562,  0.17893231,  0.79261518,
        -0.01392179,  0.34385887,  0.36800259, -0.06897634,  0.30114663,
         0.04339335, -0.13323432,  0.13481562, -0.15997395, -0.17533274,
         0.16651745,  0.01042212,  0.02770009,  0.12728798, -0.43586773,
        -0.09187035,  0.08827616, -0.6980114 , -0.15954748,  0.0390434 ,
        -0.9147734 , -0.20138982,  0.        ,  0.        ,  0.        ,
         0.03888666, -0.47333592, -0.16008905,  0.14496537, -0.72616202,
        -0.22094873,  0.20420197, -0.9234761 , -0.23718078, -0.1604946 ,
         0.05212369, -0.04848878,  0.01976278, -0.2745901 , -0.26545796,
         0.14106901, -0.0029481 , -0.24873146,  0.15021388,  0.20543793,
        -0.17611538, -0.26615071,  0.16940615, -0.10952435,  0.007667  ,
         0.03699892, -0.27983391,  0.07229839,  0.23496749, -0.27070817,
         0.01747582,  0.38211539, -0.2237262 ]])

In [205]:
t = xgb_model.predict(X_test[0].reshape(1, -1))

In [206]:
t[0]

17